# Анализ базы данных книжного сервиса

## Описание:
Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Задачи: 
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Исследовательский анализ:

Импортируем библиотеки и установим параметры для подключения к базе данных:

In [1]:
import pandas as pd
from sqlalchemy import create_engine
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) # сохраняем коннектор

Создадим функцию, выводящую результат SQL-запроса:

In [2]:
def sql_result(query):
  return pd.io.sql.read_sql(query, con = engine)

Выведем первые пять строк каждой таблицы для ознакомления:

In [3]:
query = '''

SELECT * 
FROM books
LIMIT 5

'''
sql_result(query)


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
query = '''

SELECT * 
FROM authors
LIMIT 5

'''
sql_result(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
query = '''

SELECT * 
FROM publishers
LIMIT 5

'''
sql_result(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
query = '''

SELECT * 
FROM ratings
LIMIT 5

'''
sql_result(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
query = '''

SELECT * 
FROM reviews
LIMIT 5

'''
sql_result(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


### Задача 1: посчитайте, сколько книг вышло после 1 января 2000 года:

In [8]:
query = '''

SELECT COUNT(book_id) 
FROM books
WHERE publication_date > '2000-01-01'

'''
sql_result(query)

,count
0,819


После 1 января 2000 года вышло 819 книг.

### Задача 2: для каждой книги посчитайте количество обзоров и среднюю оценку:

In [9]:
query = '''

SELECT books.title,
       COUNT (distinct review_id) as review_count,
       AVG(rating) AS avg_rating
FROM books
LEFT JOIN reviews ON reviews.book_id = books.book_id
LEFT JOIN ratings ON ratings.book_id = books.book_id
GROUP BY books.title,
         books.book_id
ORDER BY COUNT(reviews.review_id) DESC

'''
sql_result(query)  

,title,review_count,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,The Hobbit or There and Back Again,6,4.125000
2,The Catcher in the Rye,6,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
...,...,...,...
995,Essential Tales and Poems,0,4.000000
996,Leonardo's Notebooks,0,4.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


Больше всего обзоров у книги Twilight (Twilight #1) - 7

### Задача 3: определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры:

In [10]:
query = '''

SELECT publisher,
       COUNT (book_id) AS book_count
FROM books
LEFT JOIN publishers ON publishers.publisher_id=books.publisher_id
WHERE num_pages>50
GROUP BY publisher
ORDER BY book_count DESC

'''
sql_result(query)

,publisher,book_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


Издательство, выпустившее наибольшее число книг толще 50 страниц - Penguin Books. 42 книги.

### Задача 4: определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками:

In [11]:
query = '''

SELECT author, 
       AVG(rating)
FROM authors AS a
JOIN books AS b 
ON a.author_id = b.author_id
JOIN ratings AS r 
ON b.book_id = r.book_id
WHERE b.book_id IN (
SELECT book_id
FROM ratings 
GROUP BY book_id
HAVING COUNT(rating_id) >= 50
ORDER BY COUNT(rating_id) DESC)
GROUP BY author
ORDER BY avg DESC
LIMIT 1

'''
sql_result(query)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré - 4.287097

### Задача 5: посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок:

In [12]:
query = '''

SELECT AVG(review_count) as avg_review_count
FROM
(SELECT COUNT(review_id) as review_count
FROM reviews
WHERE username IN
(SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(ratings) > 50)
GROUP BY username) AS user_review_count 

'''
sql_result(query)

,avg_review_count
0,24.333333


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.

## Общие выводы:

- После 1 января 2000 года вышло 819 книг.
- Больше всего обзоров у книги Twilight (Twilight #1) - 7.
- Издательство, выпустившее наибольшее число книг толще 50 страниц - Penguin Books. 42 книги.
- Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré - 4.287097.
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.